In [1]:
#计算某个时间段的网页热度
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

conf = SparkConf()
conf.setAppName('Socket_pagehot') 
conf.setMaster('local[4]') 
sc = SparkContext(conf=conf) 
ssc = StreamingContext(sc,5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 05:43:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [2]:
pages3 = ssc.socketTextStream('localhost', 9999) 
ssc.checkpoint('file:///code/checkpoint2')

In [ ]:
def calculate_hot (line):
    words = line.split(',')
    hot = 0.1 * int(words[1]) +0.9 * int (words[2]) + 0.4 * float (words [3])+ int(words[4])
    return words[0], float(str(hot).split('.')[0] +'.' + str(hot). split('.')[1][:2])

def updata_func(new_values, last_num):
    return round(sum(new_values) + (last_num or 0), 2)

pagehot = pages3.map(lambda  line: calculate_hot(line))
pagehotsum_count = pagehot.updateStateByKey(updata_func)

pagehotsum_count.pprint(5)
ssc.start()
ssc.awaitTermination()